In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import datetime

In [2]:
nielsen15 = pd.read_csv('../../Nielsen/aggregated_nielsen_2015.csv')
nielsen16 = pd.read_csv('../..//Nielsen/aggregated_nielsen_2016.csv')
nielsen15['year'] = 2015
nielsen16['year'] = 2016

In [3]:
nielsen = pd.concat((nielsen15, nielsen16))
nielsen = nielsen[~nielsen.is_walmart]

In [4]:
## Entriy/exit dates
fandom = pd.read_csv('../data_collection/plein_de_data/fandom_traitées.csv', parse_dates=['Opening_date', 'Closing_date'])[['State', 'County_name', 'County_fips', 'Opening_date', 'Closing_date']]

# We drop the state in which we do not trust our data (some mistakes stillremain)
fandom = fandom[~np.isin(fandom.State, ('CA', 'GA', 'KS', 'LA', 'TX'))]
nielsen = nielsen[~np.isin(nielsen.store_state, ('CA', 'GA', 'KS', 'LA', 'TX'))]

# We concentrate our study on the movements (entries & exits) that happened between 2014 and 2017 (fiscal year). We add 2014 and 2017 to eliminate side effects.
movements = fandom[((fandom.Opening_date >= '2014-01-31') & (fandom.Opening_date <= '2018-01-31')) | ((fandom.Closing_date >= '2014-01-31') & (fandom.Closing_date <= '2018-01-31'))]
#movements['year'] = movements.Opening_date.dt.year
#movements['month'] = movements.Opening_date.dt.month

In [5]:
hey = nielsen.groupby('product_group_descr').count()
categories = hey[hey.is_walmart>=25000].index

In [ ]:
categories = [
    "FRESH PRODUCE",
    "BREAD AND BAKED GOODS",
    "MILK",
    "SNACKS",
    "PACKAGED MEATS-DELI",
    "CHEESE",
    "UNPREP MEAT/POULTRY/SEAFOOD-FRZN",
    "CARBONATED BEVERAGES",
    "CONDIMENTS, GRAVIES, AND SAUCES",
    "CANDY",
    "JUICE, DRINKS - CANNED, BOTTLED",
    "EGGS",
    "CEREAL",
    "PASTA",
]

In [ ]:
categories =[
    "COT CHEESE, SOUR CREAM, TOPPINGS",
    "PACKAGED MILK AND MODIFIERS",
    "SPICES, SEASONING, EXTRACTS",
    "SUGAR, SWEETENERS"
]

In [5]:
pool = pd.DataFrame(nielsen.groupby(['is_walmart', 'store_state', 'guessed_store_county', 'guessed_store_county_fips', 'purchase_year', 'purchase_month']).mean()['upc_price']).reset_index()

categories = ['_']

## PREMIER MODELE

$$Prices_{i, t} = \alpha + \beta treat_i + \gamma treat_i* post_t$$

In [12]:
# We choose to focus on milk prices
for category in categories:
    print("=========================================================")
    print(category)

    product_group = nielsen[nielsen.product_group_descr == category]


    # The control group is composed by all states where nothing (no entry nor exit) happened.
    control = product_group[~np.isin(product_group.guessed_store_county_fips, movements)].copy()
    print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")


    # The treatment group is composed by the states where one exit took place in 2016 and where this entry is the only movement
    count = movements.groupby('County_fips').count()
    count = count[count.State == 1] # No more than one movement in the treatement group
    treatment_movements = movements[(np.isin(movements.County_fips, count.index))]
    treatment_movements = treatment_movements[(treatment_movements.Closing_date>='2015-01-31' ) & (treatment_movements.Closing_date<='2017-01-31') & (treatment_movements.Opening_date<'2015-01-31')]

    treatment = product_group[np.isin(product_group.guessed_store_county_fips, treatment_movements.County_fips )].copy()
    treatment = treatment.merge(treatment_movements, left_on='guessed_store_county_fips', right_on='County_fips')
    print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")


    # We create our dummies for the regression
    control['treat'] = False
    control['interaction'] = False

    treatment['treat'] = True
    treatment['interaction'] = (treatment.purchase_month > treatment.Closing_date.dt.month) & (treatment.purchase_year >= treatment.Closing_date.dt.year)


    # Final dataset for the regression :

    df = pd.concat((control, treatment))[['upc_price', 'treat', 'interaction']]
    df = df[df.upc_price != 0]
    reg0 = smf.ols(formula='np.log(upc_price) ~ treat + interaction', data=df)
    results0 = reg0.fit()
    #if abs(results0.params[2] / results0.bse[2]) > 2. :
    if abs(results0.params[2] / results0.bse[2])>2.:
        print(f"Coef : {np.exp(results0.params[2])-1}")
        print(f"Coef/err : {abs(results0.params[2] / results0.bse[2])}")
        print(f"CI_up : {np.exp(results0.conf_int(alpha=0.05)[0][2])-1}")
        print(f"CI_down : {np.exp(results0.conf_int(alpha=0.05)[1][2])-1}")

BAKED GOODS-FROZEN
Size of the control group: 1706.
Size of the treatment group: 42.
BAKING MIXES
Size of the control group: 1735.
Size of the treatment group: 42.
BAKING SUPPLIES
Size of the control group: 1753.
Size of the treatment group: 43.
BREAD AND BAKED GOODS
Size of the control group: 1836.
Size of the treatment group: 43.
BREAKFAST FOOD
Size of the control group: 1665.
Size of the treatment group: 38.
BUTTER AND MARGARINE
Size of the control group: 1764.
Size of the treatment group: 42.
CANDY
Size of the control group: 1821.
Size of the treatment group: 42.
CARBONATED BEVERAGES
Size of the control group: 1815.
Size of the treatment group: 43.
CEREAL
Size of the control group: 1777.
Size of the treatment group: 41.
CHEESE
Size of the control group: 1816.
Size of the treatment group: 43.
COFFEE
Size of the control group: 1680.
Size of the treatment group: 40.
CONDIMENTS, GRAVIES, AND SAUCES
Size of the control group: 1821.
Size of the treatment group: 43.
COOKIES
Size of the co

## DEUXIEME MODELE - monthly time fixed effects

In [13]:
# We choose to focus on milk prices
for category in categories:
    product_group = nielsen[nielsen.product_group_descr == category]


    # The control group is composed by all states where nothing (no entry nor exit) happened.
    control = product_group[~np.isin(product_group.guessed_store_county_fips, movements)].copy()
    #print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")


    # The treatment group is composed by the states where one entry took place in 2016 and where this entry is the only movement
    count = movements.groupby('County_fips').count()
    count = count[count.State == 1] # No more than one movement in the treatement group
    treatment_movements = movements[(np.isin(movements.County_fips, count.index))]
    treatment_movements = treatment_movements[(treatment_movements.Closing_date>='2015-01-31' ) & (treatment_movements.Closing_date<='2017-01-31') & (treatment_movements.Opening_date<'2015-01-31')]

    treatment = product_group[np.isin(product_group.guessed_store_county_fips, treatment_movements.County_fips )].copy()
    treatment = treatment.merge(treatment_movements, left_on='guessed_store_county_fips', right_on='County_fips')
    #print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")


    # We create our dummies for the regression
    control['treat'] = False
    control['interaction'] = False
    control['time_effects'] = list(zip(control.purchase_year, control.purchase_month))

    treatment['treat'] = True
    treatment['interaction'] = (treatment.purchase_month > treatment.Closing_date.dt.month) & (treatment.purchase_year >= treatment.Closing_date.dt.year)
    treatment['time_effects'] = list(zip(treatment.purchase_year, treatment.purchase_month))


    # Final dataset for the regression :

    df = pd.concat((control, treatment))[['upc_price', 'treat', 'interaction', 'store_state', 'time_effects']]
    df = df[df.upc_price != 0]


    reg1 = smf.ols(formula='np.log(upc_price) ~ treat + interaction + C(time_effects)', data=df)
    results1 = reg1.fit()
    if abs(results1.params[2] / results1.bse[2])>2 :
        print("=========================================================")
        print(category)
        print(f"Coef : {np.exp(results1.params[2])-1}")
        print(f"Coef/err : {abs(results1.params[2] / results1.bse[2])}")
        print(f"CI_up : {np.exp(results1.conf_int(alpha=0.05)[0][2])-1}")
        print(f"CI_down : {np.exp(results1.conf_int(alpha=0.05)[1][2])-1}")

EGGS
Coef : 0.08370880883884824
Coef/err : 3.41044528002763
CI_up : 0.03477939089443516
CI_down : 0.13495184837395557
NUTS
Coef : -0.084933692579375
Coef/err : 2.2640382177663776
CI_up : -0.15261533359637203
CI_down : -0.011846236809799349
PACKAGED MEATS-DELI
Coef : -0.052328250036368984
Coef/err : 2.3373390532642975
CI_up : -0.09409238106638107
CI_down : -0.008638710052687681
PIZZA/SNACKS/HORS DOEURVES-FRZN
Coef : -0.08124663130802412
Coef/err : 2.7031217066428668
CI_up : -0.13599898511988495
CI_down : -0.023024582210729894
SOUP
Coef : 0.06736942122474576
Coef/err : 2.1376839169120445
CI_up : 0.005432553829368247
CI_down : 0.13312173653668613


## TROISIEME MODELE - state effects

In [15]:
# We choose to focus on milk prices
for category in categories:
    product_group = nielsen[nielsen.product_group_descr == category]


    # The control group is composed by all states where nothing (no entry nor exit) happened.
    control = product_group[~np.isin(product_group.guessed_store_county_fips, movements)].copy()
    #print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")


    # The treatment group is composed by the states where one entry took place in 2016 and where this entry is the only movement
    count = movements.groupby('County_fips').count()
    count = count[count.State == 1] # No more than one movement in the treatement group
    treatment_movements = movements[(np.isin(movements.County_fips, count.index))]
    treatment_movements = treatment_movements[(treatment_movements.Closing_date>='2015-01-31' ) & (treatment_movements.Closing_date<='2017-01-31') & (treatment_movements.Opening_date<'2015-01-31')]

    treatment = product_group[np.isin(product_group.guessed_store_county_fips, treatment_movements.County_fips )].copy()
    treatment = treatment.merge(treatment_movements, left_on='guessed_store_county_fips', right_on='County_fips')
    #print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")


    # We create our dummies for the regression
    control['treat'] = False
    control['interaction'] = False
    control['time_effects'] = list(zip(control.purchase_year, control.purchase_month))

    treatment['treat'] = True
    treatment['interaction'] = (treatment.purchase_month > treatment.Closing_date.dt.month) & (treatment.purchase_year >= treatment.Closing_date.dt.year)
    treatment['time_effects'] = list(zip(treatment.purchase_year, treatment.purchase_month))


    # Final dataset for the regression :

    df = pd.concat((control, treatment))[['upc_price', 'treat', 'interaction', 'store_state', 'time_effects']]
    df = df[df.upc_price != 0]


    reg1 = smf.ols(formula='np.log(upc_price) ~ treat + interaction + C(time_effects) + C(store_state)', data=df)
    results1 = reg1.fit()
    if abs(results1.params[2] / results1.bse[2]) >= 2.:
        print("=========================================================")
        print(category)
        print(f"Coef : {np.exp(results1.params[2])-1}")
        print(f"Coef/err : {abs(results1.params[2] / results1.bse[2])}")
        print(f"CI_up : {np.exp(results1.conf_int(alpha=0.05)[0][2])-1}")
        print(f"CI_down : {np.exp(results1.conf_int(alpha=0.05)[1][2])-1}")

EGGS
Coef : 0.06947366134762212
Coef/err : 3.0925562048098225
CI_up : 0.024901948466283352
CI_down : 0.11598374266718015
PACKAGED MEATS-DELI
Coef : -0.05590749016660068
Coef/err : 2.5966001077707728
CI_up : -0.09602915087169339
CI_down : -0.014005077726773618
PIZZA/SNACKS/HORS DOEURVES-FRZN
Coef : -0.0765301504521384
Coef/err : 2.6049658996542533
CI_up : -0.13022729572712854
CI_down : -0.019517905270565183
SOUP
Coef : 0.06987212542000099
Coef/err : 2.278690264523323
CI_up : 0.00948925523133215
CI_down : 0.13386681316227622


## QUATRIEME MODELE - state*time

In [8]:
categories

Index(['BAKED GOODS-FROZEN', 'BAKING MIXES', 'BAKING SUPPLIES',
       'BREAD AND BAKED GOODS', 'BREAKFAST FOOD', 'BUTTER AND MARGARINE',
       'CANDY', 'CARBONATED BEVERAGES', 'CEREAL', 'CHEESE', 'COFFEE',
       'CONDIMENTS, GRAVIES, AND SAUCES', 'COOKIES',
       'COT CHEESE, SOUR CREAM, TOPPINGS', 'CRACKERS',
       'DESSERTS, GELATINS, SYRUP', 'DETERGENTS', 'DOUGH PRODUCTS',
       'DRESSINGS/SALADS/PREP FOODS-DELI', 'EGGS', 'FRESH PRODUCE',
       'FRUIT - CANNED', 'ICE CREAM, NOVELTIES', 'JAMS, JELLIES, SPREADS',
       'JUICE, DRINKS - CANNED, BOTTLED', 'MEDICATIONS/REMEDIES/HEALTH AIDS',
       'MILK', 'NUTS', 'PACKAGED MEATS-DELI', 'PACKAGED MILK AND MODIFIERS',
       'PAPER PRODUCTS', 'PASTA', 'PET FOOD', 'PICKLES, OLIVES, AND RELISH',
       'PIZZA/SNACKS/HORS DOEURVES-FRZN', 'PREPARED FOOD-DRY MIXES',
       'PREPARED FOOD-READY-TO-SERVE', 'PREPARED FOODS-FROZEN',
       'SALAD DRESSINGS, MAYO, TOPPINGS', 'SHORTENING, OIL', 'SNACKS',
       'SOFT DRINKS-NON-CARBONATED', 

In [6]:
# We choose to focus on milk prices
for category in ['FRUIT - CANNED']:
    #product_group = nielsen[nielsen.product_group_descr == category]
    product_group = pool

    # The control group is composed by all states where nothing (no entry nor exit) happened.
    control = product_group[~np.isin(product_group.guessed_store_county_fips, movements)].copy()
    #print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")

    # We keep in the control group the only counties where we have data for the entire time period (24 months)
    nb_months = control.groupby('guessed_store_county_fips').count()
    control = control[np.isin(control.guessed_store_county_fips, nb_months[nb_months.is_walmart==24].index)]
    #print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")

    #nb_obs = control.groupby('guessed_store_county_fips').min()
    #control = control[np.isin(control.guessed_store_county_fips, nb_obs[nb_obs.nb_of_obs > 3].index)]
    #print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")



    # The treatment group is composed by the states where one entry took place in 2016 and where this entry is the only movement
    count = movements.groupby('County_fips').count()
    count = count[count.State == 1] # No more than one movement in the treatement group
    treatment_movements = movements[(np.isin(movements.County_fips, count.index))]
    treatment_movements = treatment_movements[(treatment_movements.Closing_date>='2015-01-31' ) & (treatment_movements.Closing_date<='2017-01-31') & (treatment_movements.Opening_date<'2015-01-31')]

    treatment = product_group[np.isin(product_group.guessed_store_county_fips, treatment_movements.County_fips )].copy()
    treatment = treatment.merge(treatment_movements, left_on='guessed_store_county_fips', right_on='County_fips')
    #print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")


    # We create our dummies for the regression
    control['treat'] = False
    control['interaction'] = False
    control['purchase_0'] = control.purchase_month + 12 * (control.purchase_year - 2015)
    control['effects'] = list(zip(control.store_state, control.purchase_0))


    treatment['treat'] = True
    treatment['purchase_0'] = treatment.purchase_month + 12 * (treatment.purchase_year - 2015)
    treatment['closing_0'] = treatment.Closing_date.dt.month  + 12 * (treatment.Closing_date.dt.year - 2015)
    treatment['interaction'] = treatment.purchase_0 >= treatment.closing_0
    treatment = treatment[abs(treatment.purchase_0-treatment.closing_0) >= 1]
    treatment['effects'] = list(zip(treatment.store_state, treatment.purchase_0))


    # Final dataset for the regression :

    df = pd.concat((control, treatment))[['upc_price', 'treat', 'interaction', 'effects']]
    df = df[df.upc_price != 0]


    reg1 = smf.ols(formula='np.log(upc_price) ~ treat + interaction + C(effects)', data=df)
    results1 = reg1.fit()
    print("=========================================================")
    print(category)
    #print(f"Coef/err : {abs(results1.params[2] / results1.bse[2])}")
    
    if abs(results1.params[2] / results1.bse[2]) >= 2.:
        print(f"Coef : {np.exp(results1.params[2])-1}")
        print(f"CI_up : {np.exp(results1.conf_int(alpha=0.05)[0][2])-1}")
        print(f"CI_down : {np.exp(results1.conf_int(alpha=0.05)[1][2])-1}")
    print(f"Coef/err : {abs(results1.params[2] / results1.bse[2])}")



FRUIT - CANNED
Coef/err : 0.23747696611987287


In [35]:
results1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      np.log(upc_price)   R-squared:                       0.156
Model:                            OLS   Adj. R-squared:                  0.131
Method:                 Least Squares   F-statistic:                     6.250
Date:                Wed, 09 Nov 2022   Prob (F-statistic):               0.00
Time:                        16:01:02   Log-Likelihood:                 945.72
No. Observations:               35887   AIC:                             176.6
Df Residuals:                   34853   BIC:                             8953.
Df Model:                        1033                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    1.1343      0.037     30.358      0.000       1.061       1.208
treat[T.True]                0.0476      0.011      4.320      0.000       0.026       0.069
interaction[T.True]         -0.0040      0.017     -0.237      0.812      -0.037       0.029
C(effects)[T.('AL', 2)]     -0.0241      0.053     -0.456      0.649      -0.128       0.079
C(effects)[T.('AL', 3)]     -0.0082      0.053     -0.155      0.877      -0.112       0.095
C(effects)[T.('AL', 4)]     -0.0142      0.053     -0.270      0.787      -0.118       0.089
C(effects)[T.('AL', 5)]     -0.0590      0.053     -1.117      0.264      -0.163       0.045
C(effects)[T.('AL', 6)]     -0.0305      0.053     -0.577      0.564      -0.134       0.073
C(effects)[T.('AL', 7)]     -0.0333      0.053     -0.631      0.528      -0.137       0.070
C(effects)[T.('AL', 8)]     -0.0632      0.053     -1.197      0.232      -0.167       0.040
C(effects)[T.('AL', 9)]     -0.0150      0.053     -0.285      0.776      -0.119       0.088
C(effects)[T.('AL', 10)]    -0.0091      0.053     -0.172      0.864      -0.113       0.094
C(effects)[T.('AL', 11)]    -0.0525      0.053     -0.995      0.320      -0.156       0.051
C(effects)[T.('AL', 12)]     0.0725      0.053      1.373      0.170      -0.031       0.176
C(effects)[T.('AL', 13)]    -0.0252      0.054     -0.465      0.642      -0.132       0.081
C(effects)[T.('AL', 14)]    -0.0054      0.053     -0.102      0.919      -0.109       0.098
C(effects)[T.('AL', 15)]    -0.0353      0.053     -0.667      0.504      -0.139       0.068
C(effects)[T.('AL', 16)]    -0.0427      0.053     -0.808      0.419      -0.146       0.061
C(effects)[T.('AL', 17)]    -0.0158      0.053     -0.299      0.765      -0.119       0.088
C(effects)[T.('AL', 18)]    -0.0457      0.053     -0.865      0.387      -0.149       0.058
C(effects)[T.('AL', 19)]    -0.0025      0.053     -0.047      0.962      -0.106       0.101
C(effects)[T.('AL', 20)]     0.0005      0.053      0.010      0.992      -0.103       0.104
C(effects)[T.('AL', 21)]    -0.0292      0.053     -0.552      0.581      -0.133       0.074
C(effects)[T.('AL', 22)]    -0.0543      0.053     -1.027      0.305      -0.158       0.049
C(effects)[T.('AL', 23)]    -0.0434      0.053     -0.822      0.411      -0.147       0.060
C(effects)[T.('AL', 24)]    -0.0117      0.053     -0.221      0.825      -0.115       0.092
C(effects)[T.('AR', 1)]     -0.0069      0.056     -0.122      0.903      -0.117       0.104
C(effects)[T.('AR', 2)]     -0.0178      0.056     -0.315      0.753      -0.128       0.093
C(effects)[T.('AR', 3)]      0.0296      0.056      0.524      0.600      -0.081       0.140
C(effects)[T.('AR', 4)]      0.0080      0.056      0.141      0.888      -0.103       0.119
C(effects)[T.(